In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from sklearn.model_selection import train_test_split

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [17]:
class ImageDataset(Dataset):
    def __init__(self, annotations_dataframe, data_path):
        self.img_labels = annotations_dataframe
        self.data_path = data_path

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = self.img_labels.iloc[idx, 1]
        image = read_image(self.data_path + '/' + img_path).type(torch.float32)
        labels = torch.tensor(self.img_labels.values[idx, 2:].astype(np.float32))
        sample = {"image": image, "labels": labels}
        return sample

In [21]:
def evaluate(model, criterion, data, epoch, train_loss=None):
    model.eval()
    print(f'\repoch: {epoch}/{NUM_EPOCHS}, evaluating model...', end='')
    losses = []
    accs = []
    for batch in data:
        with torch.no_grad():
            inputs = batch['image']
            labels = batch['labels']
            outputs = model(inputs)
            #print(outputs, labels)
            loss = criterion(outputs, labels)
            losses.append(loss.item())
            #label = np.argmax(outputs.cpu().detach().numpy(), axis=1)
            #accs.append(np.mean(labels.cpu().detach().numpy() == label))
    loss = np.mean(losses)
    #acc = np.mean(accs)
    print(f'\repoch: {epoch}/{NUM_EPOCHS}, train loss: {("%.5f"%train_loss) if train_loss else "unknown"}, val loss: {"%.5f"%loss}') #, acc: {"%.5f"%acc}')
    model.train()
    return loss#, acc

In [22]:
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.MaxPool2d(4),
    nn.Dropout2d(0.25),
    
    nn.Conv2d(64, 128, kernel_size=5, padding=2),
    nn.ReLU(),
    
    nn.Conv2d(128, 256, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(256),
    nn.MaxPool2d(2),
    nn.Dropout2d(0.25),
    
    nn.Flatten(),
    nn.Linear(65536, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 27),
).to(device)

optimizer = torch.optim.Adam(model.parameters())#, lr=0.0001, weight_decay=1e-6)
criterion = nn.BCEWithLogitsLoss().to(device)

In [ ]:
VAL_SIZE = 0.2
BATCH_SIZE = 64
NUM_EPOCHS = 100
EVAL_STEP = 1
SAVE_MODEL_IF_LOSS_IS_LESS_THAN = 1
DATA_PATH = 'emotic_clean_cropped'

train_df = pd.read_csv('labels/train.csv')
val_df = pd.read_csv('labels/val.csv')
train_data = ImageDataset(train_df, DATA_PATH)
val_data = ImageDataset(val_df, DATA_PATH)
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)

num_batches = len(train_data) // BATCH_SIZE

evaluate(model, criterion, val_dataloader, 0)

for epoch in range(1, NUM_EPOCHS + 1):
    print(f'\repoch: {epoch}/{NUM_EPOCHS}, training...', end='')
    losses = []
    i = 0
    best_loss = SAVE_MODEL_IF_LOSS_IS_LESS_THAN
    
    for batch in train_dataloader:
        i += 1
        print(f'\repoch: {epoch}/{NUM_EPOCHS}, batch: {i}/{num_batches}...', end='')
        inputs = batch['image']
        labels = batch['labels']
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        
    if epoch % EVAL_STEP == 0:
        val_loss, _ = evaluate(model, criterion, val_dataloader, epoch, np.mean(losses))
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model, f'cnn.pkl')

epoch: 0/100, train loss: unknown, val loss: 0.79959
epoch: 1/100, batch: 4/266...